In [5]:
import numpy as np
import keras
import cv2
import pandas as pd
import tensorflow as tf
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import InputLayer
from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import OneHotEncoder
import itertools
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [6]:
file_path = 'D:\Admin\Documents\Year_4\AMLS\Assessment\dataset_AMLS_20-21\celeba\img'
labels_path = 'D:\Admin\Documents\Year_4\AMLS\Assessment\dataset_AMLS_20-21\celeba\labels.csv'

In [7]:
dft = pd.read_csv(labels_path, skiprows=lambda x: x in range(1,5000,5))
dft["smiling"] = dft["smiling"].replace(to_replace=[-1], value=['Frown'])
dft["smiling"] = dft["smiling"].replace(to_replace=[1], value=['Smile'])
dft["gender"] = dft["gender"].replace(to_replace=[-1], value=['Female'])
dft["gender"] = dft["gender"].replace(to_replace=[1], value=['Male'])
one_hot_s = pd.get_dummies(dft["smiling"])
one_hot_g = pd.get_dummies(dft["gender"])
dft = dft.drop(columns=["gender", "smiling"])
dft = dft.join(one_hot_g)
dft = dft.join(one_hot_s)
datagen = ImageDataGenerator(rescale=1./255)
train_generator = datagen.flow_from_dataframe(
    dataframe=dft, 
    directory=file_path,
    x_col="img_name", 
    y_col=["Female", "Male"], 
    class_mode="raw", 
    target_size=(55,45), 
    batch_size=100, 
    color_mode='grayscale', 
    interpolation='bicubic')

Found 4000 validated image filenames.


In [8]:
dfv = pd.read_csv(labels_path, skiprows=lambda x: x not in range(0,5000,5))
dfv["smiling"] = dfv["smiling"].replace(to_replace=[-1], value=['Frown'])
dfv["smiling"] = dfv["smiling"].replace(to_replace=[1], value=['Smile'])
dfv["gender"] = dfv["gender"].replace(to_replace=[-1], value=['Female'])
dfv["gender"] = dfv["gender"].replace(to_replace=[1], value=['Male'])
one_hot_s = pd.get_dummies(dfv["smiling"])
one_hot_g = pd.get_dummies(dfv["gender"])
dfv = dfv.drop(columns=["gender", "smiling"])
dfv = dfv.join(one_hot_g)
dfv = dfv.join(one_hot_s)
datagen = ImageDataGenerator(rescale=1./255)
test_generator = datagen.flow_from_dataframe(
    dataframe=dfv, 
    directory=file_path,
    x_col="img_name", 
    y_col=["Female", "Male"], 
    class_mode="raw", 
    target_size=(55,45), 
    batch_size=100, 
    color_mode='grayscale', 
    interpolation='bicubic')

Found 999 validated image filenames.


In [9]:
model = Sequential()
model.add(InputLayer(input_shape=(55,45,1)))
model.add(Conv2D(filters=100, kernel_size=(3,3), strides=2, activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=2))
model.add(Conv2D(filters=200, kernel_size=(2,2), strides=2, activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=2))
model.add(Conv2D(filters=400,kernel_size=(2,2),activation="relu"))
model.add(Flatten())
model.add(Dense(1024))
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Dropout(0.2))
model.add(Dense(2, activation="softmax"))

In [10]:
model.compile(Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [11]:
train_steps = train_generator.n//train_generator.batch_size
test_steps = test_generator.n//test_generator.batch_size

In [12]:
model.fit_generator(generator=train_generator, 
                    steps_per_epoch=train_steps,
                    validation_data=test_generator,
                    validation_steps=test_steps,
                    epochs=25)

Epoch 1/50
40/40 [==============================] - 11s 279ms/step - loss: 0.7181 - accuracy: 0.6555 - val_loss: 0.6903 - val_accuracy: 0.6000
Epoch 2/50
40/40 [==============================] - 12s 294ms/step - loss: 0.4644 - accuracy: 0.7915 - val_loss: 0.6845 - val_accuracy: 0.5350
Epoch 3/50
40/40 [==============================] - 11s 275ms/step - loss: 0.3673 - accuracy: 0.8397 - val_loss: 0.6831 - val_accuracy: 0.5462
Epoch 4/50
40/40 [==============================] - 12s 290ms/step - loss: 0.2983 - accuracy: 0.8727 - val_loss: 0.7708 - val_accuracy: 0.5161
Epoch 5/50
40/40 [==============================] - 11s 276ms/step - loss: 0.2270 - accuracy: 0.9097 - val_loss: 0.8390 - val_accuracy: 0.5284
Epoch 6/50
40/40 [==============================] - 12s 293ms/step - loss: 0.1807 - accuracy: 0.9312 - val_loss: 0.6212 - val_accuracy: 0.5551
Epoch 7/50
40/40 [==============================] - 11s 284ms/step - loss: 0.1409 - accuracy: 0.9452 - val_loss: 0.5505 - val_accuracy: 0.6018

KeyboardInterrupt: 